## Setup

In [1]:
import numpy as np
import pandas as pd

In [2]:
import os

ENV = "kaggle"
if 'COLAB_JUPYTER_TOKEN' in os.environ:
    ENV = "colab"


In [3]:
if ENV == "colab":
    from google.colab import drive
    drive.mount('/content/drive')

    # Upload kaggle.json if not present
    import os
    if not os.path.exists('kaggle.json'):
        from google.colab import files
        uploaded = files.upload()

libraries = [
    'functools',
    'gc',
    'gzip',
    'hashlib',
    'kaggle',
    'matplotlib',
    'numpy',
    'os',
    'pandas',
    'scikit-learn',
    'sentencepiece',
    'torch',
    'tqdm',
    'transformers==4.30.2',
    'lightgbm',
    'nltk',
    'gensim',
    'bs4',
    'optuna'
]

if ENV == "colab":
    import subprocess
    for lib in libraries:
        subprocess.run(["pip", "install", lib])

In [4]:
import random
import numpy as np
import torch

def set_seed(seed_value):
    """Set seed for reproducibility."""
    random.seed(seed_value)            # Set seed for python's built-in random
    np.random.seed(seed_value)         # Set seed for numpy
    torch.manual_seed(seed_value)      # Set seed for pytorch

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)  # Use any number as seed


## Load Data

In [5]:
import pandas as pd

FILE_NAMES = ['prompts_train.csv', 'summaries_train.csv', 'prompts_test.csv', 'summaries_test.csv']

def read_files(base_path, file_names=FILE_NAMES):
    """Read multiple CSV files from a given path."""
    dataframes = []
    for file in file_names:
        dataframes.append(pd.read_csv(base_path + file))
    return tuple(dataframes)

def load_data_in_kaggle():
    """Load data when running in a Kaggle environment."""
    base_path = '/kaggle/input/commonlit-evaluate-student-summaries/'
    return read_files(base_path)

def load_data_in_colab():
    """Load data when running in a Google Colab environment."""
    base_path = "/content/drive/MyDrive/kaggle/commonlit-evaluate-student-summaries/"
    return read_files(base_path)

def load_data():
    """Load data based on the environment (Kaggle/Colab)."""
    if ENV == "kaggle":
        return load_data_in_kaggle()
    elif ENV == "colab":
        return load_data_in_colab()
    else:
        raise ValueError("Unknown environment.")

df_train_prompts, df_train_summaries, df_test_prompts, df_test_summaries = load_data()


## Merge set

In [6]:
df_train = df_train_summaries.merge(df_train_prompts, on='prompt_id')
df_test = df_test_summaries.merge(df_test_prompts, on='prompt_id')

df_test["wording"] = 0.0
df_test["content"] = 0.0

import pandas as pd
import os

if ENV == "kaggle":
    file_path = '/kaggle/input/summaries-features-2023-10-05-14-51-24/oof_predictions.csv'
    if os.path.exists(file_path):
        df_train = pd.read_csv(file_path)



print(f"df_train.shape: {df_train.shape}")
print(f"df_test.shape: {df_test.shape}")

df_train.head()

df_train.shape: (7165, 38)
df_test.shape: (4, 8)


,student_id,prompt_id,text,content,wording,prompt_question,prompt_title,prompt_text,spelling_error_count,full_text,...,summary_tokens,length_ratio,word_overlap_count,bigram_overlap_count,trigram_overlap_count,quotes_count,predicted_content,transformer_content,predicted_wording,transformer_wording
0,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an...",0,Excerpt from The Jungle [SEP] Summarize the va...,...,"['▁They', '▁would', '▁rub', '▁it', '▁up', '▁wi...",0.049252,0,22,10,0,-0.809405,-0.809405,-0.203639,-0.203639
1,0071d51dab6d,ebad26,They would use chemicals and substances to cha...,0.205683,0.380538,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an...",0,Excerpt from The Jungle [SEP] Summarize the va...,...,"['▁They', '▁would', '▁use', '▁chemicals', '▁an...",0.041337,0,1,0,0,-0.492028,-0.492028,0.092170,0.092170
2,00746c7c79c3,ebad26,"Many times the factories would, according to t...",-0.878889,-0.966330,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an...",0,Excerpt from The Jungle [SEP] Summarize the va...,...,"['▁Many', '▁times', '▁the', '▁factories', '▁wo...",0.048373,0,14,8,0,-0.915069,-0.915069,-0.751800,-0.751800
3,008db54e7cbc,ebad26,The factory covered up and used spolied meat i...,1.216547,1.166914,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an...",0,Excerpt from The Jungle [SEP] Summarize the va...,...,"['▁The', '▁factory', '▁covered', '▁up', '▁and'...",0.109059,2,33,15,0,0.721002,0.721002,0.855613,0.855613
4,00b8461e9c37,ebad26,The factory would rub up the spoiled meat with...,-1.236282,-0.285223,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an...",0,Excerpt from The Jungle [SEP] Summarize the va...,...,"['▁The', '▁factory', '▁would', '▁rub', '▁up', ...",0.041337,1,25,20,0,-0.838479,-0.838479,-0.745378,-0.745378


## Loaders


In [7]:
BATCH_SIZE = 4
DROPOUT = 0.007
MAX_GRAD_NORM = 1.0
MAX_LENGTH = 512
WEIGHT_DECAY = 0.02
LEARNING_RATE = 1.5e-5

# MODEL_NAME = 'deberta-v3-large'
MODEL_NAME = 'debertav3base'

In [8]:
from transformers import AutoModel, AutoTokenizer, AutoConfig

def get_root_path():
    if ENV == "kaggle":
        MODEL_PATH = '/kaggle/input'
        return MODEL_PATH
    elif ENV == "colab":
        MODEL_PATH = '/content/drive/MyDrive/kaggle'
        return MODEL_PATH
    else:
        raise EnvironmentError("Unknown environment.")

def load_pretrained_from_path(model_path, model_name, config_updates):
    """Load pretrained model and tokenizer from a given path."""

    tokenizer = get_tokenizer(model_name)

    full_path = f"{model_path}/{model_name}"
    config = AutoConfig.from_pretrained(full_path)
    config.update(config_updates)
    model = AutoModel.from_pretrained(full_path, config=config)

    return model, tokenizer


def load_pretrained(model_name=MODEL_NAME):
    MODEL_PATH = get_root_path()

    config_updates = {
        "hidden_dropout_prob": DROPOUT,
        "max_grad_norm": MAX_GRAD_NORM,
        "weight_decay": WEIGHT_DECAY
    }

    return load_pretrained_from_path(MODEL_PATH, model_name, config_updates)


def get_tokenizer(model_name):
    model_path = get_root_path()
    full_path = f"{model_path}/{model_name}"
    return AutoTokenizer.from_pretrained(full_path)


model, tokenizer = load_pretrained()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/p

### Helpers

In [9]:
import os
import pickle
from functools import wraps

def disk_cache_if_colab(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        # Generate a cache file name based on the function name and arguments
        cache_file = f"{func.__name__}_{'_'.join(map(str, args))}.pkl"

        # Check if ENV is set to 'colab'
        if os.environ.get('ENV') == 'colab':
            # Check if cache exists
            if os.path.exists(cache_file):
                with open(cache_file, 'rb') as f:
                    return pickle.load(f)

            # Compute the result and cache it
            result = func(*args, **kwargs)
            with open(cache_file, 'wb') as f:
                pickle.dump(result, f)
            return result
        else:
            # If ENV is not 'colab', just compute the result
            return func(*args, **kwargs)

    return wrapper


## Features

In [10]:
import re
from bs4 import BeautifulSoup
import pandas as pd

def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis

    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+')  # Removes website links
    text = template.sub(r'', text)

    soup = BeautifulSoup(text, 'lxml')  # Removes HTML tags
    only_text = soup.get_text()
    text = only_text

    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    text = re.sub(r"[^a-zA-Z\d]", " ", text) # Remove special Charecters
    text = re.sub('\n+', '\n', text)
    text = re.sub('\.+', '.', text)
    text = re.sub(' +', ' ', text) # Remove Extra Spaces

    return text

# df_train['cleaned_text'] = df_train['text'].apply(text_cleaning)
# df_test['cleaned_text'] = df_test['text'].apply(text_cleaning)


In [11]:
# https://www.kaggle.com/code/atamazian/spell-checker-using-word2vec-updated-work-by-cpmp/notebook

if ENV == "kaggle":
    import gensim
    import re
    import pandas as pd
    from collections import Counter
    # Load the model (assuming model path is provided)
    model_path = '/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin'
    model = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

    # Create a set for faster lookup
    known_words_set = set(model.index_to_key)

# Tokenization function
def tokenize(text):
    return re.findall(r'\b\w+\b', text.lower())

# Known function with caching
known_cache = {}
def known(word):
    if word in known_cache:
        return known_cache[word]

    is_known = word in known_words_set
    known_cache[word] = is_known
    return is_known

# Count spelling errors in a text string
def count_spelling_errors(text):
    tokens = tokenize(text)
    errors = sum(1 for word in tokens if not known(word))
    return errors

if ENV == "colab":
    df_train['spelling_error_count'] = 0
    df_test['spelling_error_count'] = 0
if ENV == "kaggle":
    df_train['spelling_error_count'] = df_train['text'].apply(lambda x: count_spelling_errors(x))
    df_test['spelling_error_count'] = df_test['text'].apply(lambda x: count_spelling_errors(x))

In [12]:
import pandas as pd
import gzip

def calculate_compressed_size(text):
    text_bytes = text.encode('utf-8')
    compressed_bytes = gzip.compress(text_bytes)
    return len(compressed_bytes)

def add_custom_features(df):
    separator_token = ' [SEP] '
    df['full_text'] = df["prompt_title"] + separator_token + df['prompt_question'] + separator_token + df['text']

    # Create the "word_count" feature
    df['word_count'] = df['text'].apply(lambda x: len(str(x).split()))

    # Create the "lexical_similarity" feature
    df['lexical_similarity'] = df.apply(
        lambda row: 1 - calculate_compressed_size(row['text'] + row['prompt_question']) / (calculate_compressed_size(row['text']) + calculate_compressed_size(row['prompt_question'])),
        axis=1
    )

    # Create the "semantic_similarity" feature if it doesn't exist
    if 'semantic_similarity' not in df.columns:
        df['semantic_similarity'] = 0.0

add_custom_features(df_train)
add_custom_features(df_test)

In [13]:
def ngrams(tokens, n):
    return [tuple(tokens[i:i + n]) for i in range(len(tokens) - n + 1)]

@disk_cache_if_colab
def rouge_n(hypothesis, reference, n=1):
    hyp_ngrams = set(ngrams(hypothesis.split(), n))
    ref_ngrams = set(ngrams(reference.split(), n))
    overlap = len(hyp_ngrams.intersection(ref_ngrams))
    precision = overlap / len(hyp_ngrams) if hyp_ngrams else 0
    recall = overlap / len(ref_ngrams) if ref_ngrams else 0
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    return {"f1": f1, "precision": precision, "recall": recall}

def lcs(X, Y):
    m, n = len(X), len(Y)
    L = [[0] * (n + 1) for i in range(m + 1)]
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0:
                L[i][j] = 0
            elif X[i - 1] == Y[j - 1]:
                L[i][j] = L[i - 1][j - 1] + 1
            else:
                L[i][j] = max(L[i - 1][j], L[i][j - 1])
    return L[m][n]

@disk_cache_if_colab
def rouge_l(hypothesis, reference):
    lcs_length = lcs(hypothesis.split(), reference.split())
    precision = lcs_length / len(hypothesis.split()) if hypothesis else 0
    recall = lcs_length / len(reference.split()) if reference else 0
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    return {"f1": f1, "precision": precision, "recall": recall}

def skip_bigrams(tokens):
    return [(tokens[i], tokens[j]) for i in range(len(tokens)) for j in range(i+1, len(tokens))]

@disk_cache_if_colab
def rouge_s(hypothesis, reference):
    hyp_skip_bigrams = set(skip_bigrams(hypothesis.split()))
    ref_skip_bigrams = set(skip_bigrams(reference.split()))
    overlap = len(hyp_skip_bigrams.intersection(ref_skip_bigrams))
    precision = overlap / len(hyp_skip_bigrams) if hyp_skip_bigrams else 0
    recall = overlap / len(ref_skip_bigrams) if ref_skip_bigrams else 0
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    return {"f1": f1, "precision": precision, "recall": recall}

def add_scores(row):
    hyp = row['text']
    ref = row['prompt_text']
    scores_1 = rouge_n(hyp, ref, 1)
    scores_2 = rouge_n(hyp, ref, 2)
    scores_l = rouge_l(hyp, ref)
    scores_s = rouge_s(hyp, ref)
    return pd.Series({
        'rouge_1_f1': scores_1['f1'],
        'rouge_2_f1': scores_2['f1'],
        'rouge_l_f1': scores_l['f1'],
        'rouge_s_f1': scores_s['f1'],
        'rouge_1_precision': scores_1['precision'],
        'rouge_2_precision': scores_2['precision'],
        'rouge_l_precision': scores_l['precision'],
        'rouge_s_precision': scores_s['precision'],
        'rouge_1_recall': scores_1['recall'],
        'rouge_2_recall': scores_2['recall'],
        'rouge_l_recall': scores_l['recall'],
        'rouge_s_recall': scores_s['recall'],
    })


if ENV == "colab":
    df_train = pd.concat([df_train, df_train.apply(add_scores, axis=1)], axis=1)

df_test = pd.concat([df_test, df_test.apply(add_scores, axis=1)], axis=1)


In [14]:
ROUGE_COLUMNS = [
    'rouge_1_f1', 'rouge_1_precision', 'rouge_1_recall',
    'rouge_2_f1', 'rouge_2_precision', 'rouge_2_recall',
    'rouge_l_f1', 'rouge_l_precision', 'rouge_l_recall',
    'rouge_s_f1', 'rouge_s_precision', 'rouge_s_recall'
]

In [15]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from tqdm import tqdm
tqdm.pandas()
import re

class Preprocessor:
    def __init__(self,
                model_name: str,
                ) -> None:
        self.tokenizer = get_tokenizer(model_name)
        self.STOP_WORDS = set(stopwords.words('english'))

    def count_text_length(self, df: pd.DataFrame, col:str) -> pd.Series:
        """ text length """
        tokenizer=self.tokenizer
        return df[col].progress_apply(lambda x: len(tokenizer.encode(x)))

    def word_overlap_count(self, row):
        """ intersection(prompt_text, text) """
        def check_is_stop_word(word):
            return word in self.STOP_WORDS

        prompt_words = row['prompt_tokens']
        summary_words = row['summary_tokens']
        if self.STOP_WORDS:
            prompt_words = list(filter(check_is_stop_word, prompt_words))
            summary_words = list(filter(check_is_stop_word, summary_words))
        return len(set(prompt_words).intersection(set(summary_words)))

    def ngrams(self, token, n):
        # Use the zip function to help us generate n-grams
        # Concatentate the tokens into ngrams and return
        ngrams = zip(*[token[i:] for i in range(n)])
        return [" ".join(ngram) for ngram in ngrams]

    def ngram_co_occurrence(self, row, n: int):
        # Tokenize the original text and summary into words
        original_tokens = row['prompt_tokens']
        summary_tokens = row['summary_tokens']

        # Generate n-grams for the original text and summary
        original_ngrams = set(self.ngrams(original_tokens, n))
        summary_ngrams = set(self.ngrams(summary_tokens, n))

        # Calculate the number of common n-grams
        common_ngrams = original_ngrams.intersection(summary_ngrams)

        # # Optionally, you can get the frequency of common n-grams for a more nuanced analysis
        # original_ngram_freq = Counter(ngrams(original_words, n))
        # summary_ngram_freq = Counter(ngrams(summary_words, n))
        # common_ngram_freq = {ngram: min(original_ngram_freq[ngram], summary_ngram_freq[ngram]) for ngram in common_ngrams}

        return len(common_ngrams)


    def quotes_count(self, row):
        summary = row['text']
        text = row['prompt_text']
        quotes_from_summary = re.findall(r'"([^"]*)"', summary)
        if len(quotes_from_summary)>0:
            return [quote in text for quote in quotes_from_summary].count(True)
        else:
            return 0

    def run(self, dataframe: pd.DataFrame) -> pd.DataFrame:

        dataframe["prompt_length"] = dataframe["prompt_text"].apply(
            lambda x: len(self.tokenizer.encode(x))
        )
        dataframe["prompt_tokens"] = dataframe["prompt_text"].apply(
            lambda x: self.tokenizer.convert_ids_to_tokens(
                self.tokenizer.encode(x),
                skip_special_tokens=True
            )
        )

        dataframe["summary_length"] = dataframe["text"].apply(
            lambda x: len(self.tokenizer.encode(x))
        )
        dataframe["summary_tokens"] = dataframe["text"].apply(
            lambda x: self.tokenizer.convert_ids_to_tokens(
                self.tokenizer.encode(x),
                skip_special_tokens=True
            )

        )

        dataframe['length_ratio'] = dataframe['summary_length'] / dataframe['prompt_length']

        dataframe['word_overlap_count'] = dataframe.progress_apply(self.word_overlap_count, axis=1)
        dataframe['bigram_overlap_count'] = dataframe.progress_apply(
            self.ngram_co_occurrence,args=(2,), axis=1
        )
        dataframe['trigram_overlap_count'] = dataframe.progress_apply(
            self.ngram_co_occurrence, args=(3,), axis=1
        )

        dataframe['quotes_count'] = dataframe.progress_apply(self.quotes_count, axis=1)

        return dataframe.drop(columns=["summary_tokens", "prompt_tokens"])

preprocessor = Preprocessor(model_name="debertav3base")

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
if ENV == "colab":
    train = preprocessor.run(df_train)

test = preprocessor.run(df_test)

test.head()

100%|██████████| 4/4 [00:00<00:00, 2675.79it/s]


,student_id,prompt_id,text,prompt_question,prompt_title,prompt_text,wording,content,spelling_error_count,full_text,...,rouge_2_recall,rouge_l_recall,rouge_s_recall,prompt_length,summary_length,length_ratio,word_overlap_count,bigram_overlap_count,trigram_overlap_count,quotes_count
0,000000ffffff,abc123,Example text 1,Summarize...,Example Title 1,Heading\nText...,0.0,0.0,0,Example Title 1 [SEP] Summarize... [SEP] Examp...,...,0.0,0.0,0.0,7,5,0.714286,0,0,0,0
1,222222cccccc,abc123,Example text 3,Summarize...,Example Title 1,Heading\nText...,0.0,0.0,0,Example Title 1 [SEP] Summarize... [SEP] Examp...,...,0.0,0.0,0.0,7,5,0.714286,0,0,0,0
2,111111eeeeee,def789,Example text 2,Summarize...,Example Title 2,Heading\nText...,0.0,0.0,0,Example Title 2 [SEP] Summarize... [SEP] Examp...,...,0.0,0.0,0.0,7,5,0.714286,0,0,0,0
3,333333dddddd,def789,Example text 4,Summarize...,Example Title 2,Heading\nText...,0.0,0.0,0,Example Title 2 [SEP] Summarize... [SEP] Examp...,...,0.0,0.0,0.0,7,5,0.714286,0,0,0,0


In [17]:
ALL_JAPANESE_FEATURES = ["quotes_count", "trigram_overlap_count", "bigram_overlap_count", "word_overlap_count", "length_ratio", "prompt_length", "summary_length"]


In [18]:
import torch

if torch.cuda.is_available():
    n_gpu = torch.cuda.device_count()
    device_ids = list(range(n_gpu))
    device = torch.device(f'cuda:{device_ids[0]}')  # Choose the first device
    print('There are %d GPU(s) available.' % n_gpu)
    print('We will use the GPUs:', device_ids)
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
    device_ids = None

There are 2 GPU(s) available.
We will use the GPUs: [0, 1]


In [19]:
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity
import hashlib
from functools import lru_cache
import torch.nn as nn

semantic_model, tokenizer = load_pretrained(model_name="debertav3base")
semantic_model.to(device)

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    semantic_model = nn.DataParallel(semantic_model)

maxsize = None if ENV == "colab" else 128
@disk_cache_if_colab
@lru_cache(maxsize=maxsize)
def get_embedding(text):
    inputs = tokenizer(text, max_length=MAX_LENGTH, truncation=True, padding='max_length', return_tensors='pt')

    if device_ids:
        inputs = {name: tensor.to(device) for name, tensor in inputs.items()}  # move inputs to GPU
    with torch.no_grad():
        outputs = semantic_model(**inputs)

    embedding = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    return embedding

# Compute embeddings and similarity score
def get_similarity(row):
    prompt_embedding = get_embedding(row['prompt_text'])
    text_embedding = get_embedding(row['text'])
    similarity = cosine_similarity(prompt_embedding, text_embedding)
    return similarity[0][0]

# Add similarity score to DataFrame
if ENV == "colab":
    df_train['semantic_similarity'] = df_train.apply(get_similarity, axis=1)

df_test['semantic_similarity'] = df_test.apply(get_similarity, axis=1)

print("Done!")

import gc
import torch

del semantic_model
gc.collect()
torch.cuda.empty_cache()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at /kaggle/input/debertav3base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.bias', 'mask_predictions.classifier.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm

Using 2 GPUs!
Done!


## Cross Validation

In [20]:
from sklearn.model_selection import KFold
import pandas as pd

def split_dataframe_by_prompt(df, n_splits=4):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    combined_train_dfs = []
    combined_val_dfs = []

    unique_prompts = df['prompt_id'].unique()

    for train_prompt_ids, val_prompt_ids in kf.split(unique_prompts):
        train_dataframes = [df[df['prompt_id'] == unique_prompts[id]] for id in train_prompt_ids]
        val_dataframes = [df[df['prompt_id'] == unique_prompts[id]] for id in val_prompt_ids]

        combined_train_dfs.append(pd.concat(train_dataframes))
        combined_val_dfs.append(pd.concat(val_dataframes))

    return combined_train_dfs, combined_val_dfs

## Loss

In [21]:
import numpy as np

def compute_mcrmse(df):
    def rmse(actual, predictions):
        return np.sqrt(np.mean((actual - predictions)**2))

    rmse_content = rmse(df['content'], df['predicted_content'])
    rmse_wording = rmse(df['wording'], df['predicted_wording'])

    mcrmse = np.mean([rmse_content, rmse_wording])

    return mcrmse, rmse_content, rmse_wording

## Training

In [22]:
import torch
from torch import nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.nn.parameter import Parameter

class TextDataset(Dataset):
    def __init__(self, df, tokenizer, max_length, target):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.target = target

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        text = row['full_text']
        inputs = self.tokenizer.encode_plus(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        inputs['input_ids'] = inputs['input_ids'].squeeze()
        inputs['attention_mask'] = inputs['attention_mask'].squeeze()
        return inputs, torch.tensor([row[self.target]], dtype=torch.float)


class GeMText(nn.Module):
    def __init__(self, dim = 1, p=3, eps=1e-6):
        super(GeMText, self).__init__()
        self.dim = dim
        self.p = Parameter(torch.ones(1) * p)
        self.eps = eps
        self.feat_mult = 1

    def forward(self, last_hidden_state, attention_mask):
        attention_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.shape)
        x = (last_hidden_state.clamp(min=self.eps) * attention_mask_expanded).pow(self.p).sum(self.dim)
        ret = x / attention_mask_expanded.sum(self.dim).clip(min=self.eps)
        ret = ret.pow(1 / self.p)
        return ret

# https://raphaelb.org/posts/freezing-bert/
def partial_freeze(model, frozen_count=4):
    for param in model.encoder.layer[:frozen_count].parameters():
        param.requires_grad = False
    for param in model.encoder.layer[frozen_count:].parameters():
            param.requires_grad = True

import torch
import torch.nn as nn
# Assuming GeMText is imported or defined elsewhere

class RegressionModel(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base_model = base_model
        # Safely retrieve the hidden size
        embedding_size = getattr(base_model.config, 'hidden_size', None)
        if embedding_size is None:
            raise ValueError(f"'hidden_size' not found in config of model {type(base_model)}")

        self.gem_pooler = GeMText()

        # Additional dense layer and dropout
        self.dense = nn.Linear(embedding_size, embedding_size)
        self.dropout = nn.Dropout(0.1)  # Set the dropout rate according to your needs

        self.regressor = nn.Linear(embedding_size, 1) ## only one target per model
        self._init_weights(self.regressor)
        self._init_weights(self.dense)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(self, **inputs):
        outputs = self.base_model(**inputs)
        sentence_embedding = self.gem_pooler(outputs.last_hidden_state, inputs['attention_mask'])

        # Extra dense layer and dropout
        sentence_embedding = self.dense(sentence_embedding)
        sentence_embedding = torch.tanh(sentence_embedding)
        sentence_embedding = self.dropout(sentence_embedding)

        return self.regressor(sentence_embedding)


class MCRMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss(reduction='none')  # We will handle the mean computation manually

    def forward(self, pred, target):
        squared_errors = self.mse(pred, target)
        rmse_per_column = torch.sqrt(torch.mean(squared_errors, dim=0))
        mcrmse = torch.mean(rmse_per_column)
        return mcrmse

In [23]:
def predict_with_transformer(model, tokenizer, dataframe, batch_size=4):
    """Use the transformer model to make predictions in batches."""

    # Set the model to evaluation mode
    model.eval()

    texts = dataframe['full_text'].tolist()
    total_samples = len(texts)

    predictions = []

    for i in range(0, total_samples, batch_size):
        inputs = tokenizer(texts[i:i+batch_size], truncation=True, padding="max_length", return_tensors="pt", max_length=MAX_LENGTH)

        # Send all inputs to device (e.g., GPU)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)

        predictions.extend(outputs[:, 0].cpu().numpy())

    return predictions



In [24]:
def compute_rmse(actual, predictions):
        return np.sqrt(np.mean((actual - predictions)**2))

def print_loss(model, tokenizer, dataframe, target):
    dataframe[f'predicted_{target}'] = predict_with_transformer(model, tokenizer, dataframe)
    rmse = compute_rmse(dataframe[target], dataframe[f'predicted_{target}'])
    print(f"{target} rmse: {rmse}")
    return rmse


In [25]:
import torch
import torch.optim as optim
from tqdm import tqdm
from torch.nn.utils import clip_grad_norm_
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import DataLoader


def layerwise_lr(optimizer, factor=0.97):
    lrs = [group['lr'] for group in optimizer.param_groups]
    for i in range(len(lrs) - 1, 0, -1):
        lrs[i-1] = lrs[i] * factor
    for i, param_group in enumerate(optimizer.param_groups):
        param_group['lr'] = lrs[i]

from copy import deepcopy

def train_phase(model, dataloader, epochs, frozen_count, target, df_val=None, early_stopping_patience=30, val_frequency=100, lr=LEARNING_RATE):
    # Initial layerwise lr setup
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=WEIGHT_DECAY)
    layerwise_lr(optimizer)  # Call this just once after initializing the optimizer
    partial_freeze(model.base_model, frozen_count=frozen_count)
    total_steps = len(dataloader) * epochs
    warmup_steps = int(total_steps * 0.1)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)
    loss_history = []
    running_loss = 0.0
    loss_func = MCRMSELoss()  # Assume this is defined elsewhere
    no_improve_count = 0
    best_val_loss = float('inf')
    best_model_state = None

    for epoch in range(epochs):
        model.train()
        print('lr', optimizer.param_groups[0]['lr'])
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")

        for i, batch in enumerate(progress_bar):
            if i % val_frequency == 0 and df_val is not None:
                val_loss = print_loss(model, tokenizer, df_val, target=target)  # Assume print_loss is defined and returns validation loss
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_model_state = deepcopy(model.state_dict())
                    no_improve_count = 0
                else:
                    no_improve_count += 1

                if no_improve_count >= early_stopping_patience:
                    print("Early stopping triggered")
                    model.load_state_dict(best_model_state)
                    return loss_history, best_model_state

            optimizer.zero_grad()
            inputs, labels = batch
            inputs = {k: v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)
            outputs = model(**inputs)
            loss = loss_func(outputs, labels)
            loss.backward()
            clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
            optimizer.step()
            scheduler.step()
            current_loss = loss.item()
            running_loss += current_loss
            loss_history.append(current_loss)

            if i % 10 == 9:
                avg_loss = running_loss / 10
                progress_bar.set_description(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
                running_loss = 0.0


    model.load_state_dict(best_model_state)
    return loss_history, best_model_state



def train_model(df_train, df_val, target):
    base_model, tokenizer = load_pretrained()  # Assume this is defined elsewhere
    base_model.config.hidden_dropout_prob = DROPOUT
    base_model.config.attention_probs_dropout_prob = DROPOUT
    model = RegressionModel(base_model)  # Assume this is defined elsewhere
    model.to(device)
    dataset_train = TextDataset(df_train, tokenizer, max_length=MAX_LENGTH, target=target)  # Assume this is defined elsewhere
    dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)

    loss_history_phase1 = train_phase(model, dataloader_train, target=target, epochs=1, frozen_count=24, df_val=df_val, lr=1e-5)
    # loss_history_phase2 = train_phase(model, dataloader_train, target=target, epochs=5, frozen_count=4, df_val=df_val, lr=1e-5)
    # loss_history_phase3 = train_phase(model, dataloader_train, target=target, epochs=2, frozen_count=0, df_val=df_val, lr=1e-6)

    loss_history = []
    # loss_history = loss_history_phase1 + loss_history_phase2 + loss_history_phase3
    return model, loss_history


In [26]:
import json
import os
from datetime import datetime
import shutil

def create_kaggle_dataset(file_paths, dataset_name):
    if ENV != "colab":
        print("Environment is not Colab. Exiting function.")
        return

    YOUR_USERNAME = "louissanna"
    now = datetime.now()
    formatted_date = now.strftime("%Y-%m-%d-%H-%M-%S")

    # List of shell commands for initial setup
    commands = [
        "pip install kaggle",
        "mkdir -p ~/.kaggle",
        "cp kaggle.json ~/.kaggle/",
        "chmod 600 ~/.kaggle/kaggle.json",
    ]

    # Only create 'my_temp_dir' if it doesn't exist
    if os.path.exists('my_temp_dir'):
        shutil.rmtree('my_temp_dir')
    if not os.path.exists('my_temp_dir'):
        commands.append("mkdir my_temp_dir")

    # Move each file in file_paths to 'my_temp_dir' if it's not already there
    for file_path in file_paths:
        filename = os.path.basename(file_path)
        if not os.path.exists(f'my_temp_dir/{filename}'):
            commands.append(f"cp {file_path} my_temp_dir/")

    # Execute each initial setup command
    for command in commands:
        get_ipython().system(command)

    # Check if 'dataset-metadata.json' doesn't exist, then set it
    if not os.path.exists("my_temp_dir/dataset-metadata.json"):
        metadata = {
            "title": dataset_name + "-" + formatted_date,
            "id": YOUR_USERNAME + "/" + dataset_name + "-" + formatted_date,
            "licenses": [{"name": "CC0-1.0"}]
        }

        with open("my_temp_dir/dataset-metadata.json", "w") as f:
            json.dump(metadata, f, indent=4)

    # Create the dataset on Kaggle
    get_ipython().system("kaggle datasets create -p my_temp_dir/")


In [27]:
# import matplotlib.pyplot as plt

# if ENV == "colab":
#     # Plotting the loss history
#     plt.figure(figsize=(10, 5))
#     plt.plot(train_loss, label='Training Loss')
#     plt.title('Training Loss Over Time')
#     plt.xlabel('Every Batche')
#     plt.ylabel('Average Loss')
#     plt.legend()
#     plt.show()

## Inference

In [28]:
import torch
import numpy as np

if ENV == "kaggle":
    # Initialize arrays to store predictions
    transformer_content_preds = []
    transformer_wording_preds = []
    for target in ["content", "wording"]:
        for fold in range(4):
            base_model, _ = load_pretrained()
            model = RegressionModel(base_model).to(device)
            model.load_state_dict(torch.load(f'/kaggle/input/tuned-deberta-2023-10-05-14-45-56/tuned_deberta_target_{target}_fold_{fold}.pth', map_location=device), strict=False) # WARNING: strict false may hide bug!

            if torch.cuda.device_count() > 1:
                model = torch.nn.DataParallel(model)

            model.eval()

            # Get predictions for the current fold
            predictions = predict_with_transformer(model, tokenizer, df_test)

            # Append the predictions to the storage arrays
            if target == "content":
                transformer_content_preds.append(predictions)
            if target == "wording":
                transformer_wording_preds.append(predictions)

    # Average the predictions across all folds
    avg_transformer_content = np.mean(transformer_content_preds, axis=0)
    avg_transformer_wording = np.mean(transformer_wording_preds, axis=0)

    # Store the averaged predictions back into df_test
    df_test['transformer_content'] = avg_transformer_content
    df_test['transformer_wording'] = avg_transformer_wording


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at /kaggle/input/debertav3base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.bias', 'mask_predictions.classifier.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm

## 4 fold training

In [29]:
def cross_validation_training(df_train, tokenizer, target):
    train_4_fold_dfs, val_4_fold_dfs = split_dataframe_by_prompt(df_train)
    oof_predictions = pd.DataFrame() # Initialize an empty DataFrame to store OOF predictions

    for fold_number in range(4):
        print(f"Fold {fold_number}")

        df_4_fold_train = train_4_fold_dfs[fold_number]
        df_4_fold_val = val_4_fold_dfs[fold_number]

        model, _ = train_model(df_4_fold_train, df_4_fold_val, target=target)
        df_4_fold_val[f'predicted_{target}'] = predict_with_transformer(model, tokenizer, df_4_fold_val)

        oof_predictions = pd.concat([oof_predictions, df_4_fold_val]) # Add the predictions to OOF DataFrame

        torch.save(model.state_dict(), f'drive/MyDrive/kaggle/colab/tuned_deberta_target_{target}_fold_{fold_number}.pth')

        import gc
        del model ; gc.collect()
        torch.cuda.empty_cache()

    return oof_predictions, None

if ENV == "colab":
    content_oof_predictions, _ = cross_validation_training(df_train, tokenizer, "content")
    content_oof_predictions["transformer_content"] = content_oof_predictions["predicted_content"]
    oof_predictions = content_oof_predictions
    wording_oof_predictions, _ = cross_validation_training(df_train, tokenizer, "wording")
    oof_predictions["predicted_wording"] = wording_oof_predictions["predicted_wording"]
    oof_predictions["transformer_wording"] = oof_predictions["predicted_wording"]
    print(compute_mcrmse(oof_predictions))
    oof_predictions.to_csv('drive/MyDrive/kaggle/colab/oof_predictions.csv', index=False)

#A verage loss across all folds:
# (0.5530839327091395, 0.4790311210744894, 0.6271367443437894)


In [30]:
if ENV == "colab":
    dataset_name = 'tuned-deberta'

    create_kaggle_dataset([
        f'drive/MyDrive/kaggle/colab/tuned_deberta_target_content_fold_0.pth',
        f'drive/MyDrive/kaggle/colab/tuned_deberta_target_content_fold_1.pth',
        f'drive/MyDrive/kaggle/colab/tuned_deberta_target_content_fold_2.pth',
        f'drive/MyDrive/kaggle/colab/tuned_deberta_target_content_fold_3.pth',
        f'drive/MyDrive/kaggle/colab/tuned_deberta_target_wording_fold_0.pth',
        f'drive/MyDrive/kaggle/colab/tuned_deberta_target_wording_fold_1.pth',
        f'drive/MyDrive/kaggle/colab/tuned_deberta_target_wording_fold_2.pth',
        f'drive/MyDrive/kaggle/colab/tuned_deberta_target_wording_fold_3.pth',
        ], dataset_name)

In [31]:
file_path = 'drive/MyDrive/kaggle/colab/oof_predictions.csv'
dataset_name = 'summaries-features'

if ENV == 'colab':
    create_kaggle_dataset([file_path], dataset_name)

## LGBM

In [32]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


# Features you want to scale
features_to_scale = ["word_count", "semantic_similarity", "lexical_similarity"] + ROUGE_COLUMNS + ALL_JAPANESE_FEATURES + ["spelling_error_count"]

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler only on the training data for the selected features
df_train[features_to_scale] = scaler.fit_transform(df_train[features_to_scale])

if ENV == "kaggle":
    # Transform the test data using the same scaler for the selected features
    df_test[features_to_scale] = scaler.transform(df_test[features_to_scale])


In [33]:
import optuna
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import numpy as np

STUDY = False
NEW_BEST_PARAMS = []
BEST_PARAMS = [{'learning_rate': 0.053299937729367544, 'max_depth': 2, 'reg_alpha': 0.07451696990115007, 'reg_lambda': 0.009270808126495363, 'min_child_samples': 9, 'num_leaves': 44, 'colsample_bytree': 0.5695112694440388, 'min_split_gain': 0.006870947135562017, 'bagging_fraction': 0.6554055041526674, 'bagging_freq': 22, 'cat_smooth': 19.32975188216686, 'n_estimators': 10000}, {'learning_rate': 0.05871433697099421, 'max_depth': 4, 'reg_alpha': 0.0723535934361538, 'reg_lambda': 0.09292286759045244, 'min_child_samples': 9, 'num_leaves': 90, 'colsample_bytree': 0.9170216958412755, 'min_split_gain': 0.017342097749941, 'bagging_fraction': 0.7227636725890166, 'bagging_freq': 18, 'cat_smooth': 51.91743263275726, 'n_estimators': 10000}, {'learning_rate': 0.09124792952318513, 'max_depth': 4, 'reg_alpha': 0.029910947952987307, 'reg_lambda': 0.07835173106783282, 'min_child_samples': 18, 'num_leaves': 121, 'colsample_bytree': 0.6760691772744497, 'min_split_gain': 0.049499519718647036, 'bagging_fraction': 0.7310634532276173, 'bagging_freq': 11, 'cat_smooth': 49.144198981461535, 'n_estimators': 10000}, {'learning_rate': 0.05898528960130003, 'max_depth': 2, 'reg_alpha': 0.03973007527529854, 'reg_lambda': 0.06818844794635358, 'min_child_samples': 5, 'num_leaves': 58, 'colsample_bytree': 0.8472983955793129, 'min_split_gain': 0.030697020587999436, 'bagging_fraction': 0.5092906139753608, 'bagging_freq': 43, 'cat_smooth': 1.0931999259387557, 'n_estimators': 10000}, {'learning_rate': 0.047794926183337076, 'max_depth': 4, 'reg_alpha': 0.07406153642264005, 'reg_lambda': 0.029737443499766383, 'min_child_samples': 10, 'num_leaves': 69, 'colsample_bytree': 0.8857146349750208, 'min_split_gain': 0.0025160672022544227, 'bagging_fraction': 0.9477595365961514, 'bagging_freq': 4, 'cat_smooth': 24.109767395139666, 'n_estimators': 10000}, {'learning_rate': 0.06529379349321789, 'max_depth': 4, 'reg_alpha': 0.0708670581253487, 'reg_lambda': 0.08636969211777444, 'min_child_samples': 8, 'num_leaves': 71, 'colsample_bytree': 0.9694972992285958, 'min_split_gain': 0.03138133894399527, 'bagging_fraction': 0.77184992555264, 'bagging_freq': 14, 'cat_smooth': 11.97678154840548, 'n_estimators': 10000}, {'learning_rate': 0.09578279722785232, 'max_depth': 3, 'reg_alpha': 0.0390992085479984, 'reg_lambda': 0.06967421136855949, 'min_child_samples': 18, 'num_leaves': 41, 'colsample_bytree': 0.7775111776969843, 'min_split_gain': 0.09114842815432107, 'bagging_fraction': 0.9515678047449987, 'bagging_freq': 39, 'cat_smooth': 37.63745398958648, 'n_estimators': 10000}, {'learning_rate': 0.06957103519487434, 'max_depth': 3, 'reg_alpha': 0.057996863416740964, 'reg_lambda': 0.04175826595010838, 'min_child_samples': 12, 'num_leaves': 83, 'colsample_bytree': 0.9476248826122162, 'min_split_gain': 0.04495623589643558, 'bagging_fraction': 0.7354351545766017, 'bagging_freq': 41, 'cat_smooth': 18.463381492601126, 'n_estimators': 10000}]

def objective(trial, train_df, val_df, feature_columns, target_column):
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'n_jobs': 2,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'max_depth': trial.suggest_int('max_depth', 2, 4),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 0.1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 0.1),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 20),
        'num_leaves': trial.suggest_int('num_leaves', 15, 127),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.1),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 50),
        'cat_smooth': trial.suggest_float('cat_smooth', 1.0, 100.0),
        'n_estimators': 10000  # Fixed n_estimators but early stopping is enabled
    }

    model = lgb.LGBMRegressor(**param)

    early_stopping_callback = lgb.early_stopping(stopping_rounds=30)

    model.fit(train_df[feature_columns], train_df[target_column],
              eval_set=[(val_df[feature_columns], val_df[target_column])],
              callbacks=[early_stopping_callback],
              eval_metric='rmse'
             )

    preds = model.predict(val_df[feature_columns])
    rmse = np.sqrt(mean_squared_error(val_df[target_column], preds))

    return rmse

def train_models_with_folds(train_folds, val_folds, feature_columns, target_column, init_column):
    models = []
    total_rmse = 0

    for train_df, val_df in zip(train_folds, val_folds):
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective(trial, train_df, val_df, feature_columns, target_column), n_trials=250)
        fig = optuna.visualization.plot_param_importances(study)
        fig.show()
        best_params = study.best_params

        best_params["n_estimators"] = 10000  # Fixed n_estimators but early stopping is enabled
        print(f"Best trial parameters: {best_params}")

        model = lgb.LGBMRegressor(**best_params)

        early_stopping_callback = lgb.early_stopping(stopping_rounds=30)

        model.fit(train_df[feature_columns], train_df[target_column],
                  eval_set=[(val_df[feature_columns], val_df[target_column])],
                  callbacks=[early_stopping_callback],
                  eval_metric='rmse'
                 )

        models.append(model)

        preds = model.predict(val_df[feature_columns])
        rmse = np.sqrt(mean_squared_error(val_df[target_column], preds))
        total_rmse += rmse

    avg_rmse = total_rmse / len(train_folds)
    print(f"Average RMSE over all folds: {avg_rmse}")

    return models, avg_rmse


def predict_with_models(models, df, feature_columns, init_column):
    init_score_test = df[init_column].values
    # predictions = [model.predict(df[feature_columns]) + init_score_test for model in models]
    predictions = [model.predict(df[feature_columns]) for model in models]
    return np.mean(predictions, axis=0)


train_4_fold_dfs, val_4_fold_dfs = split_dataframe_by_prompt(df_train)

ALL_FEATURES = ["word_count", "semantic_similarity", "lexical_similarity"] + ROUGE_COLUMNS + ALL_JAPANESE_FEATURES + ["spelling_error_count"]

# RMK: transformer_content col will be exploited by the model as initial pred
BEST_CONTENT_FEATURES = ['semantic_similarity', 'rouge_l_precision', 'rouge_2_precision', 'rouge_1_precision', 'trigram_overlap_count']
content_feature_columns = BEST_CONTENT_FEATURES + ["transformer_content"] + ["spelling_error_count", "lexical_similarity", "word_count"]
content_models, content_avg_rmse = train_models_with_folds(train_4_fold_dfs, val_4_fold_dfs, content_feature_columns, "content", "transformer_content")
if STUDY is False:
    df_test["content"] = predict_with_models(content_models, df_test, content_feature_columns, "transformer_content")

# RMK: transformer_wording col will be exploited by the model as initial pred
BEST_WORDING_FEATURES = ['semantic_similarity', 'rouge_l_precision', 'rouge_s_precision', 'rouge_1_precision', 'rouge_2_precision', 'trigram_overlap_count']
wording_feature_columns = BEST_WORDING_FEATURES + ["transformer_wording"] + ["spelling_error_count"]
content_models, wording_avg_rmse = train_models_with_folds(train_4_fold_dfs, val_4_fold_dfs, wording_feature_columns, "wording", "transformer_wording")
if STUDY is False:
    df_test["wording"] = predict_with_models(content_models, df_test, wording_feature_columns, "transformer_wording")

print(NEW_BEST_PARAMS)

print("CV", (wording_avg_rmse + content_avg_rmse) / 2)
# Content: 0.43167609356227665
# Wording: 0.5855942285573787
# CV 0.5086515987174854
# CV 0.507289558437018 (10 min_child_samples)
# CV 0.5060638266922417 (remove word count)
# CV0.504125476050318 (no gzip)
# Transformer with Title
# CV 0.5067787302165025
# CV 0.502561407241191 (optuna)
# CV 0.49726933333978357 optuna with many more hparam
# CV 0.49704301520873906
# CV 0.49714550048973666

[I 2023-10-05 15:28:17,412] A new study created in memory with name: no-name-33d135da-8881-499f-af4c-4284b68a4cfb
[I 2023-10-05 15:28:17,528] Trial 0 finished with value: 0.3892819415876798 and parameters: {'learning_rate': 0.06908470600369003, 'max_depth': 2, 'reg_alpha': 0.09699051057240059, 'reg_lambda': 0.0984373948545659, 'min_child_samples': 10, 'num_leaves': 124, 'colsample_bytree': 0.7443054171906055, 'min_split_gain': 0.06807727671637794, 'bagging_fraction': 0.5066928750212326, 'bagging_freq': 30, 'cat_smooth': 40.874228999308556}. Best is trial 0 with value: 0.3892819415876798.


[LightGBM] [Warning] bagging_fraction is set=0.5066928750212326, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5066928750212326
[LightGBM] [Warning] bagging_freq is set=30, subsample_freq=0 will be ignored. Current value: bagging_freq=30
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[148]	valid_0's rmse: 0.389282
[LightGBM] [Warning] bagging_fraction is set=0.7703379154727569, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7703379154727569
[LightGBM] [Warning] bagging_freq is set=15, subsample_freq=0 will be ignored. Current value: bagging_freq=15
Training until validation scores don't improve for 30 rounds


[I 2023-10-05 15:28:17,708] Trial 1 finished with value: 0.3979288841817379 and parameters: {'learning_rate': 0.030358979582153334, 'max_depth': 4, 'reg_alpha': 0.016015570634658007, 'reg_lambda': 0.03841687817423213, 'min_child_samples': 5, 'num_leaves': 102, 'colsample_bytree': 0.9871681463512747, 'min_split_gain': 0.042498418198332104, 'bagging_fraction': 0.7703379154727569, 'bagging_freq': 15, 'cat_smooth': 37.616332625803715}. Best is trial 0 with value: 0.3892819415876798.
[I 2023-10-05 15:28:17,789] Trial 2 finished with value: 0.38929276960608566 and parameters: {'learning_rate': 0.08307331153314965, 'max_depth': 2, 'reg_alpha': 0.0617401308782573, 'reg_lambda': 0.08916122958436194, 'min_child_samples': 16, 'num_leaves': 70, 'colsample_bytree': 0.6769109770028856, 'min_split_gain': 0.03158603281902379, 'bagging_fraction': 0.7309734035078288, 'bagging_freq': 20, 'cat_smooth': 14.504578590785705}. Best is trial 0 with value: 0.3892819415876798.
[I 2023-10-05 15:28:17,849] Trial 3

Early stopping, best iteration is:
[142]	valid_0's rmse: 0.397929
[LightGBM] [Warning] bagging_fraction is set=0.7309734035078288, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7309734035078288
[LightGBM] [Warning] bagging_freq is set=20, subsample_freq=0 will be ignored. Current value: bagging_freq=20
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[138]	valid_0's rmse: 0.389293
[LightGBM] [Warning] bagging_fraction is set=0.8160347176347087, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8160347176347087
[LightGBM] [Warning] bagging_freq is set=45, subsample_freq=0 will be ignored. Current value: bagging_freq=45
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[71]	valid_0's rmse: 0.394997
[LightGBM] [Warning] bagging_fraction is set=0.8182321546689416, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8182321546689416
[LightGBM] [Warning]

[I 2023-10-05 15:28:17,945] Trial 4 finished with value: 0.3911191781685058 and parameters: {'learning_rate': 0.04751335869628714, 'max_depth': 3, 'reg_alpha': 0.07754876307236344, 'reg_lambda': 0.039766123066376446, 'min_child_samples': 17, 'num_leaves': 51, 'colsample_bytree': 0.8298549346888869, 'min_split_gain': 0.06544959132569794, 'bagging_fraction': 0.8182321546689416, 'bagging_freq': 49, 'cat_smooth': 96.49631603104451}. Best is trial 0 with value: 0.3892819415876798.


...

[I 2023-10-05 15:37:05,808] Trial 247 finished with value: 0.6240824349584191 and parameters: {'learning_rate': 0.0965192201799388, 'max_depth': 2, 'reg_alpha': 0.0019547754222746655, 'reg_lambda': 0.047019040210043876, 'min_child_samples': 6, 'num_leaves': 17, 'colsample_bytree': 0.5948160898880077, 'min_split_gain': 0.09874239871057816, 'bagging_fraction': 0.7990777303795685, 'bagging_freq': 10, 'cat_smooth': 51.375771709786044}. Best is trial 121 with value: 0.6068794080231363.
[I 2023-10-05 15:37:05,972] Trial 248 finished with value: 0.6166392121135661 and parameters: {'learning_rate': 0.06958542415634378, 'max_depth': 2, 'reg_alpha': 0.07788311884920025, 'reg_lambda': 0.042273674934676354, 'min_child_samples': 5, 'num_leaves': 20, 'colsample_bytree': 0.964539715686746, 'min_split_gain': 0.05011799066653541, 'bagging_fraction': 0.8151153946814496, 'bagging_freq': 6, 'cat_smooth': 61.85941653878088}. Best is trial 121 with value: 0.6068794080231363.


Early stopping, best iteration is:
[138]	valid_0's rmse: 0.624082
[LightGBM] [Warning] bagging_fraction is set=0.8151153946814496, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8151153946814496
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[112]	valid_0's rmse: 0.616639


[I 2023-10-05 15:37:06,126] Trial 249 finished with value: 0.6116501715378809 and parameters: {'learning_rate': 0.08321250231004304, 'max_depth': 2, 'reg_alpha': 0.07899335055976087, 'reg_lambda': 0.038336922965471656, 'min_child_samples': 5, 'num_leaves': 22, 'colsample_bytree': 0.9441182037660063, 'min_split_gain': 0.04794390084476292, 'bagging_fraction': 0.7463118176330142, 'bagging_freq': 4, 'cat_smooth': 57.551402673190395}. Best is trial 121 with value: 0.6068794080231363.


[LightGBM] [Warning] bagging_fraction is set=0.7463118176330142, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7463118176330142
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[83]	valid_0's rmse: 0.61165


Best trial parameters: {'learning_rate': 0.09677315671859095, 'max_depth': 2, 'reg_alpha': 0.09634153434410263, 'reg_lambda': 0.04270729355058827, 'min_child_samples': 5, 'num_leaves': 26, 'colsample_bytree': 0.9413792990968098, 'min_split_gain': 0.02857061421419199, 'bagging_fraction': 0.7779851221775124, 'bagging_freq': 7, 'cat_smooth': 52.98137268211235, 'n_estimators': 10000}
[LightGBM] [Warning] bagging_fraction is set=0.7779851221775124, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7779851221775124
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[93]	valid_0's rmse: 0.606879	valid_0's l2: 0.368303
Average RMSE over all folds: 0.5579852320263243
[]
CV 0.487139721007844


## Submit

In [34]:
print(f"df_test.shape: {df_test.shape}")

def is_valid_float(x):
    return isinstance(x, float) and x == x  # This checks that x is not NaN since NaN != NaN in Python.

# Replace non-float values in 'wording' and 'content' with zero
cols_to_check = ['wording', 'content']
df_test[cols_to_check] = df_test[cols_to_check].applymap(lambda x: x if is_valid_float(x) else 0.0)

df_test[['student_id', 'content', 'wording']].to_csv('submission.csv',index=False)
display(pd.read_csv('submission.csv'))

df_test.shape: (4, 36)


,student_id,content,wording
0,000000ffffff,-1.155742,-0.649347
1,222222cccccc,-1.155742,-0.745621
2,111111eeeeee,-1.155742,-0.742798
3,333333dddddd,-1.155742,-0.753235


In [35]:
if ENV == "colab":
    from google.colab import runtime
    runtime.unassign()